# In this notebook we reproduce the experimental analysis on artificial data

### import necessary libraries and define functions for our experiments

In [1]:
from scheduling_functions import *
from scheduling_algorithms import *
from numpy import std
import sys
import copy
from random import sample, randint, seed, random
from math import isclose, ceil, floor, e, sqrt
from statistics import mean
from decimal import *
from fractions import *
import matplotlib.pyplot as plt
from operator import add

In [2]:
#creates a bounded random walk:

def random_walk_creation(num_jobs, step_size, random_seed, m, M):
    seed(random_seed)

    ws = [0]*num_jobs
    ws[0] = randint(m,M)
    steps = [randint(-step_size,step_size) for i in range(1,num_jobs)]
    for i in range(1, num_jobs):
        ws[i] = ws[i-1] + steps[i-1]
        ws[i] = min(ws[i], M)
        ws[i] = max(ws[i], m)
    return ws

In [3]:
#creates a job instance given a list of weights and T

def job_instance_creation(ws, T):
    # dictionary: key --> job id
    #            value --> (weight, release time , deadline)
    J = {}
    job_id = 1
    i = 0
    for job_weight in ws:
        J[job_id] = (job_weight , i, i+T)
        i+=1
        job_id+=1
    return J

In [4]:
#returns the energy ratio AVR_energy/Optimal_energy

def AVR_energy_ratio(_J, alpha):
    
    J = copy.deepcopy(_J)
    #speed list of average rate
    AVR_speed_list = Avg_rate(J)
    #energy consumption of AVR
    energy_AVR = compute_energy(AVR_speed_list, alpha)
    
    J = copy.deepcopy(_J)
    #speed list of the optimal schedule
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    #energy consumption of the optimal schedule
    energy_optimal = compute_energy(optimal_alg_speed_list, alpha)  
    
    return float(energy_AVR)/energy_optimal    

In [5]:
#returns the energy ratio OA_energy/Optimal_energy

def OA_energy_ratio(_J, alpha):
    
    J = copy.deepcopy(_J)
    #speed list of Optimal Available
    OA_speed_list = OptimalOnline(J)
    #energy consumption of Optimal Available
    energy_OA = sum([s**alpha for s in OA_speed_list])
    
    J = copy.deepcopy(_J)
    #speed list of the optimal schedule
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    #energy consumption of the optimal schedule
    energy_optimal = compute_energy(optimal_alg_speed_list, alpha)    
   
    return float(energy_OA)/energy_optimal

In [6]:
#returns the energy ratio BKP_energy/Optimal_energy

def BKP_energy_ratio(_J, granularity, alpha):
    
    J = copy.deepcopy(_J)
    #energy consumption of the BKP algorithm
    energy_BKP = BKP_alg(J, granularity, alpha)

    J = copy.deepcopy(_J)
    #speed list of the optimal schedule
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    #energy consumption of the optimal schedule
    energy_optimal = compute_energy(optimal_alg_speed_list, alpha)  
    
    return float(energy_BKP)/energy_optimal    

In [7]:
#returns the energy ratio LAS_energy/Optimal_energy

def LAS_energy_ratio(_J_true, _J_pred, epsilon, alpha):
    
    #compute speedlist returned by LAS
    J_true = copy.deepcopy(_J_true)
    J_pred = copy.deepcopy(_J_pred)
    online_alg_with_predictions_speed_list = Alg_with_Predictions(J_pred, J_true, epsilon)
    
    #compute speedlist of the optimal schedule of the true instance
    J_true = copy.deepcopy(_J_true)
    J_pred = copy.deepcopy(_J_pred)
    optimal_alg_speed_list, _ = Optimal_Alg(J_true)
    
    #energy computation
    energy_online_alg_with_predictions = compute_energy(online_alg_with_predictions_speed_list, alpha)
    energy_optimal = compute_energy(optimal_alg_speed_list, alpha)
    
    return float(energy_online_alg_with_predictions)/energy_optimal    

### we set the parameters of the experiments

In [8]:
# instance length and T
num_jobs = 200
T = 20

num_of_experiments = 5


step_size = 5
M = 80
m = 20

# alpha parameter of the energy consumption
alpha = 3

# time granularity for BKP algorithm
BKP_granularity = 0.25

# robustness parameters epsilon which will be tested
epsilons=[Fraction(1,100), Fraction(5,100), Fraction(10,100), Fraction(15,100), Fraction(20,100)]

### to increase reproducibility we perform experiments on the same set of (random) true instances with fixed seeds

In [9]:
J_true_lst = []
w_true_lst = []
for j in range(0,num_of_experiments):
     #create a random walk
    w_true = random_walk_creation(num_jobs, step_size=5, random_seed=j, M= M, m= m)
    
    w_true_lst.append(w_true)
    #job instance creation
    J_true = job_instance_creation(w_true, T)
    
    J_true_lst.append(J_true)

### Online Algorithms tested

(1) Average Rate Heuristic (AVR)

(2) Optimal Available (OA)

(3) Bansal, Kimbrel and Pruhs algorithm (BKP)


In [10]:
y_AVR = []
y_BKP = []
y_OA = []
dummy_y_AVR = []
dummy_y_BKP = []
dummy_y_OA = []
for j in range(0,num_of_experiments):
    J_true = J_true_lst[j]
    
    AVR = AVR_energy_ratio(J_true,alpha)
    BKP = BKP_energy_ratio(J_true,BKP_granularity, alpha)
    OA = OA_energy_ratio(J_true, alpha)
    dummy_y_AVR.append(AVR)
    dummy_y_BKP.append(BKP)
    dummy_y_OA.append(OA)
std_AVR = std(dummy_y_AVR)
std_BKP = std(dummy_y_BKP)
std_OA  = std(dummy_y_OA)
y_AVR.append(mean(dummy_y_AVR))
y_BKP.append(mean(dummy_y_BKP))
y_OA.append(mean(dummy_y_OA))

print("AVR: ", y_AVR[-1])
print("Std ", std_AVR)
print("=====================")   
print("BKP: ", y_BKP[-1])
print("Std ", std_BKP)
print("=====================")
print("OA: ", y_OA[-1])
print("Std ", std_OA)
print("=====================")


AVR:  1.2827963027467046
Std  0.060146621411726345
BKP:  8.158935369920101
Std  0.7250266771899275
OA:  1.2158973075494246
Std  0.07144926173652723


### (1) Accurate predictor

#### We create the artificial predictions of our "Accurate predictor"

In [11]:
J_pred_lst = []
for j in range(0,num_of_experiments):
    w_true = w_true_lst[j]
    
    seed(j)
    error = [randint(-step_size, step_size) for _ in range(0,num_jobs)]
    
    w_pred = list(map(add, w_true, error))
    
    
    #jon instance creation
    J_pred = job_instance_creation(w_pred, T)
    
    J_pred_lst.append(J_pred)


#### We test the performance of the Learning Augmented Scheduling (LAS) algorithm when combined with an "Accurate predictor"

In [12]:
for epsilon in epsilons:
    print("EPSILON = ", epsilon)
    y_LA_scheduling = []
    
    dummy_y_LA_schedulling = []
    for j in range(0,num_of_experiments):
        J_true = J_true_lst[j]
        J_pred = J_pred_lst[j]

        LA_scheduling = LAS_energy_ratio(J_true, J_pred, epsilon, alpha)

        dummy_y_LA_schedulling.append(LA_scheduling)
    
    y_LA_scheduling.append(mean(dummy_y_LA_schedulling))
    std_LA_scheduling = std(dummy_y_LA_schedulling)
    
    print("LA scheduling: ", y_LA_scheduling[-1])
    print("Std of LA scheduling ", std_LA_scheduling)
    print("=====================")   

EPSILON =  1/100
LA scheduling:  1.008797775717588
Std of LA scheduling  0.004224472127155453
EPSILON =  1/20
LA scheduling:  1.008797775717588
Std of LA scheduling  0.004224472127155453
EPSILON =  1/10
LA scheduling:  1.0375354569452877
Std of LA scheduling  0.0082099358605688
EPSILON =  3/20
LA scheduling:  1.0611072203024607
Std of LA scheduling  0.012049543589772636
EPSILON =  1/5
LA scheduling:  1.0877195788197458
Std of LA scheduling  0.01610979841702974


### (2) Random predictor

#### we create the artificial predictions of our "Random predictor"

In [13]:
J_pred_lst = []
for j in range(0,num_of_experiments):
    seed(j)
    error = [randint(-step_size, step_size) for _ in range(0,num_jobs)]
    
    w_pred = [randint(m,M) for _ in range(0,num_jobs)]
    
    
    #jon instance creation
    J_pred = job_instance_creation(w_pred, T)
    
    J_pred_lst.append(J_pred)

#### We test the performance of the Learning Augmented Scheduling (LAS) algorithm when combined with a "Random predictor"

In [14]:
for epsilon in epsilons:
    print("EPSILON = ", epsilon)
    y_LA_scheduling = []
    
    dummy_y_LA_schedulling = []
    for j in range(0,num_of_experiments):
        J_true = J_true_lst[j]
        J_pred = J_pred_lst[j]

        LA_scheduling = LAS_energy_ratio(J_true, J_pred, epsilon, alpha)

        dummy_y_LA_schedulling.append(LA_scheduling)

    y_LA_scheduling.append(mean(dummy_y_LA_schedulling))
    std_LA_scheduling = std(dummy_y_LA_schedulling)
    
    print("LA scheduling: ", y_LA_scheduling[-1])
    print("Std of LA scheduling ", std_LA_scheduling)

    print("=====================")   

EPSILON =  1/100
LA scheduling:  1.2272868504934065
Std of LA scheduling  0.15257040402039085
EPSILON =  1/20
LA scheduling:  1.2272868504934065
Std of LA scheduling  0.15257040402039085
EPSILON =  1/10
LA scheduling:  1.2142136836796331
Std of LA scheduling  0.09960649189584056
EPSILON =  3/20
LA scheduling:  1.21758096564505
Std of LA scheduling  0.07884581292267899
EPSILON =  1/5
LA scheduling:  1.2227424844570534
Std of LA scheduling  0.06910965561802387


### Misleading predictor

#### We create the artificial predictions of our "Misleading predictor"

In [15]:
J_pred_lst = []
for j in range(0,num_of_experiments):
    w_true = w_true_lst[j]
    
    w_pred = []
    for i in range(0,num_jobs):
        w_pred.append((M-w_true[i]) + m)
    
    
    #jon instance creation
    J_pred = job_instance_creation(w_pred, T)
    
    J_pred_lst.append(J_pred)

#### We test the performance of the Learning Augmented Scheduling (LAS) algorithm when combined with a "Misleading predictor"

In [16]:
for epsilon in epsilons:
    print("EPSILON = ", epsilon)
    y_LA_scheduling = []
    
    dummy_y_LA_schedulling = []
    for j in range(0,num_of_experiments):
        J_true = J_true_lst[j]
        J_pred = J_pred_lst[j]

        LA_scheduling = LAS_energy_ratio(J_true, J_pred, epsilon, alpha)

        dummy_y_LA_schedulling.append(LA_scheduling)

    y_LA_scheduling.append(max(dummy_y_LA_schedulling))
    std_LA_scheduling = std(dummy_y_LA_schedulling)
    
    print("LA scheduling: ", y_LA_scheduling[-1])
    print("Std of LA scheduling ", std_LA_scheduling)

    print("=====================")   

EPSILON =  1/100
LA scheduling:  1.7664395341654813
Std of LA scheduling  0.18229572573459152
EPSILON =  1/20
LA scheduling:  1.7664395341654813
Std of LA scheduling  0.18229572573459152
EPSILON =  1/10
LA scheduling:  1.7216664219682838
Std of LA scheduling  0.1666494939496596
EPSILON =  3/20
LA scheduling:  1.6865635651495268
Std of LA scheduling  0.15017607269079586
EPSILON =  1/5
LA scheduling:  1.6317485187164829
Std of LA scheduling  0.14190036190566915
